In [1]:
import itertools as it
import pickle as pkl
import os
from collections import OrderedDict
%run ../src/sampling_utils.py
%run ../src/shared_utils.py

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
disease = "covid19"
prediction_region = "germany"

with open('../data/counties/counties.pkl', "rb") as f:
    counties = pkl.load(f)

data = load_daily_data(disease, prediction_region, counties)
data = data.iloc[:, 0:31]
times = uniform_times_by_day(data.index)
locs = uniform_locations_by_county(counties)

def temporal_bfs(x): return bspline_bfs(x, np.array(
    [0, 0, 1, 2, 3, 4, 5]) * 24 * 3600.0, 2)

def spatial_bfs(x): return [gaussian_bf(x, σ)
                            for σ in [6.25, 12.5, 25.0, 50.0]]

num_features = len(temporal_bfs(tt.fmatrix("tmp"))) * len(spatial_bfs(tt.fmatrix("tmp")))

In [3]:
# %%timeit
# ia_bfs = build_ia_bfs(temporal_bfs, spatial_bfs)
# res_new = sample_ia_effects(data, times, locs, ia_bfs, num_features)

In [4]:
# %%timeit
# samp = IAEffectSampler(data, times, locs, temporal_bfs, spatial_bfs, verbose=False)
# res_old = samp(data.index, data.columns)

In [5]:
ia_bfs = build_ia_bfs(temporal_bfs, spatial_bfs)
res_new = sample_ia_effects(data, times, locs, ia_bfs, num_features)
res_new

TypeError: Bad input argument to theano function with name "/home/pascal/src/BSTIM-Covid19/src/sampling_utils.py:356" at index 3 (0-based).  
Backtrace when that variable is created:

  File "/home/pascal/src/BSTIM-Covid19/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2858, in run_cell
    raw_cell, store_history, silent, shell_futures)
  File "/home/pascal/src/BSTIM-Covid19/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2886, in _run_cell
    return runner(coro)
  File "/home/pascal/src/BSTIM-Covid19/.venv/lib/python3.6/site-packages/IPython/core/async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)
  File "/home/pascal/src/BSTIM-Covid19/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3063, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "/home/pascal/src/BSTIM-Covid19/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 325

In [ ]:
samp = IAEffectSampler(data, times, locs, temporal_bfs, spatial_bfs, verbose=False)
res_old = samp(data.index, data.columns)
res_old

In [ ]:
np.array_equal(res_old, res_new)